In [17]:
import numpy as np

def gaussQuadStd1d(g, noOfIntegPt):
    """Computes the integral ∫ g(ξ) dξ using Gaussian quadrature on [-1,1]"""
    if noOfIntegPt == 2:
        points = np.array([-1/np.sqrt(3), 1/np.sqrt(3)])
        weights = np.array([1, 1])
    elif noOfIntegPt == 3:
        points = np.array([-np.sqrt(3/5), 0, np.sqrt(3/5)])
        weights = np.array([5/9, 8/9, 5/9])
    else:
        return 0
    
    return sum(w * g(xi) for w, xi in zip(weights, points))

def gaussQuad1d(fn, lowerLimit, upperLimit, noOfIntegPt):
    transform = lambda ξ: (upperLimit - lowerLimit) / 2 * ξ + (upperLimit + lowerLimit) / 2
    g = lambda ξ: fn(transform(ξ)) * (upperLimit - lowerLimit) / 2 

    return gaussQuadStd1d(g, noOfIntegPt)

def shapeFn1d(i, x, x1, x2, p):
    
    if x < x1 or x > x2:
        return 0  
    
    if p == 1:  
        if i == 1:
            return (x2 - x) / (x2 - x1)
        elif i == 2:
            return (x - x1) / (x2 - x1)
    
    elif p == 2: 
        mid = (x1 + x2) / 2  
        
        if i == 1:
            return (x - mid) * (x - x2) / ((x1 - mid) * (x1 - x2))
        elif i == 2:
            return (x - x1) * (x - x2) / ((mid - x1) * (mid - x2))
        elif i == 3:
            return (x - x1) * (x - mid) / ((x2 - x1) * (x2 - mid))
    
    return 0  


def shapeFnDer1d(i, x, x1, x2, p):
    
    if x < x1 or x > x2:
        return 0  
    
    if p == 1:  
        if i == 1:
            return -1 / (x2 - x1)
        elif i == 2:
            return 1 / (x2 - x1)
    
    elif p == 2:  
        mid = (x1 + x2) / 2  
        
        if i == 1:
            return ((2*x - x2 - mid) / ((x1 - mid) * (x1 - x2)))
        elif i == 2:
            return ((2*x - x1 - x2) / ((mid - x1) * (mid - x2)))
        elif i == 3:
            return ((2*x - x1 - mid) / ((x2 - x1) * (x2 - mid)))
    
    return 0
def meij(e, i, j, xh, shapeFn, noOfIntegPt):
    """Computes the mass matrix entry M^e_ij."""
    x1, x2 = xh[e-1], xh[e]
    
    def integrand(x):
        return shapeFn1d(i, x, x1, x2, shapeFn) * shapeFn1d(j, x, x1, x2, shapeFn)

    return gaussQuad1d(integrand, x1, x2, noOfIntegPt)

def keij(a, c, e, i, j, xh, shapeFn, noOfIntegPt):
    """Computes the stiffness matrix entry K^e_ij."""
    x1, x2 = xh[e-1], xh[e]

    def integrand(x):
        dphi_i = shapeFnDer1d(i, x, x1, x2, shapeFn)
        dphi_j = shapeFnDer1d(j, x, x1, x2, shapeFn)
        phi_i = shapeFn1d(i, x, x1, x2, shapeFn)
        phi_j = shapeFn1d(j, x, x1, x2, shapeFn)

        return (a(x) * dphi_i * dphi_j + c(x) * phi_i * phi_j)

    return gaussQuad1d(integrand, x1, x2, noOfIntegPt)

def fei(a, c, f, p0, e, i, xh, shapeFn, noOfIntegPt, t):
    """Computes the load vector entry F^e_i."""
    x1, x2 = xh[e-1], xh[e]

    def integrand(x):
        phi_i = shapeFn1d(i, x, x1, x2, shapeFn)
        dphi_i = shapeFnDer1d(i, x, x1, x2, shapeFn)
        G = p0*shapeFn1d(1,x,xh[0],xh[1],shapeFn)
        Gx = p0*shapeFnDer1d(1,x,xh[0],xh[1],shapeFn)
        return (f(x,t) * shapeFn1d(i, x, x1, x2, shapeFn) - a(x)*Gx*dphi_i - c(x)*G*phi_i)
        
    return gaussQuad1d(integrand, x1, x2, noOfIntegPt)

In [2]:
import numpy as np
import scipy.sparse as sp

def massM(xh, shapeFn, noOfIntegPt):
    
    n_elem = len(xh) - 1  # Number of elements
    n_nodes = n_elem * shapeFn  # Total nodes based on shape function order
    
    # Initialize sparse matrix
    M = sp.lil_matrix((n_nodes, n_nodes))
    
    # Loop over each element
    for e in range(1, n_elem + 1):
        x1, x2 = xh[e-1], xh[e]  # Element endpoints
        
        # Local mass matrix for element e
        local_M = np.zeros((shapeFn + 1, shapeFn + 1))
        
        for i in range(1, shapeFn + 2):
            for j in range(1, shapeFn + 2):
                local_M[i-1, j-1] = meij(e, i, j, xh, shapeFn, noOfIntegPt)
        
        # Assemble into global matrix
        if shapeFn == 1:
            if e == 1:
                M[0,0] = local_M[-1,-1]
            else:
                for i in range(shapeFn + 1):
                    for j in range(shapeFn + 1):
                        global_i = e - 2 + i
                        global_j = e - 2 + j
                        M[global_i, global_j] += local_M[i, j]
        else:
            if e == 1:
                for i in range(1,3):
                    for j in range(1,3):
                        M[i-1,j-1] = local_M[i,j]
            else:
                for i in range(shapeFn + 1):
                    for j in range(shapeFn + 1):
                        global_i = 2*e - 3 + i        
                        global_j = 2*e - 3 + j
                        M[global_i, global_j] += local_M[i,j]
    
    return M.tocsr()  # Convert to CSR format for efficiency


In [3]:
import numpy as np
import scipy.sparse as sp

def stiffK(a, c, xh, shapeFn, noOfIntegPt):
    
    n_elem = len(xh) - 1  # Number of elements
    n_nodes = n_elem * shapeFn  # Total nodes based on shape function order

    # Initialize sparse matrix
    K = sp.lil_matrix((n_nodes, n_nodes))

    # Loop over each element
    for e in range(1, n_elem + 1):
        x1, x2 = xh[e-1], xh[e]  # Element endpoints
        
        # Local stiffness matrix for element e
        local_K = np.zeros((shapeFn + 1, shapeFn + 1))
        
        for i in range(1, shapeFn + 2):
            for j in range(1, shapeFn + 2):
                local_K[i-1, j-1] = keij(a, c, e, i, j, xh, shapeFn, noOfIntegPt)

        # Assemble into global matrix
        if shapeFn == 1:
            if e == 1:
                K[0,0] = local_K[-1,-1]
            else:
                for i in range(shapeFn + 1):
                    for j in range(shapeFn + 1):
                        global_i = e - 2 + i
                        global_j = e - 2 + j
                        K[global_i, global_j] += local_K[i, j]
        else:
            if e == 1:
                for i in range(1,3):
                    for j in range(1,3):
                        K[i-1,j-1] = local_K[i,j]
            else:
                for i in range(shapeFn + 1):
                    for j in range(shapeFn + 1):
                        global_i = 2*e - 3 + i        
                        global_j = 2*e - 3 + j
                        K[global_i, global_j] += local_K[i,j]

    return K.tocsr()  # Convert to CSR format for efficient calculations


In [20]:
import numpy as np

def loadF(a, c, f, p0, QL, xh, shapeFn, noOfIntegPt):
    
    n_elem = len(xh) - 1  # Number of elements
    n_nodes = n_elem * shapeFn  # Total nodes based on shape function order
    
    def F_func(t):
        # Initialize global load vector
        F = np.zeros(n_nodes)

        # Loop over elements
        for e in range(1, n_elem + 1):
            x1, x2 = xh[e-1], xh[e]  # Element endpoints

            # Local load vector
            local_F = np.zeros(shapeFn + 1)

            for i in range(1, shapeFn + 2):
                local_F[i-1] = fei(a, c, f, p0, e, i, xh, shapeFn, noOfIntegPt, t)

            print(local_F)

            # Assemble local contributions into global load vector
            if shapeFn == 1:
                if e == 1:
                    F[0] = local_F[-1]
                else:
                    for i in range(shapeFn + 1):
                        global_i = e - 2 + i
                        F[global_i] += local_F[i]
            else:
                if e == 1:
                    for i in range (1,3):
                        F[i-1] = local_F[i]
                else:
                    for i in range(shapeFn + 1):
                        global_i = 2*e - 3 + i
                        F[global_i] += local_F[i]

        # Apply Neumann boundary condition at x = L
        F[-1] += QL(t)
        return F
    return F_func


In [9]:
import numpy as np
from scipy.sparse.linalg import spsolve

def crank_nicolson_solver(xh, a, c, f, p0, QL, shapeFn, noOfIntegPt, u0_func, t0, T, dt):
    """
    Solves the time-dependent PDE using the Crank–Nicolson method in 1D FEM.
    
    Parameters:
    - xh: mesh nodes
    - a, c, f: functions of x (and t for f)
    - p0: Dirichlet BC at x = 0 (u(0,t))
    - QL: Neumann BC at x = L
    - shapeFn: 1 or 2
    - noOfIntegPt: Gaussian quadrature points (e.g., 2 or 3)
    - u0_func: initial condition function u0(x)
    - t0, T: initial and final time
    - dt: time step
    
    Returns:
    - uh_list: list of solutions uh at each time step
    - time: array of time steps
    """
    # Number of nodes
    n_elem = len(xh) - 1
    if shapeFn == 1:
        n_nodes = n_elem + 1
    elif shapeFn == 2:
        n_nodes = 2 * n_elem + 1
    else:
        raise ValueError("shapeFn must be 1 or 2")

    # Mass and stiffness matrices
    M = massM(xh, shapeFn, noOfIntegPt)
    K = stiffK(a, c, xh, shapeFn, noOfIntegPt)

    # Time discretization
    time = np.arange(t0, T + dt, dt)
    N = len(time)

    # Load vector function F(t)
    F_func = loadF(a, c, f, p0, QL, xh, shapeFn, noOfIntegPt)

    # Initial condition
    uh = np.array([u0_func(x) for x in np.linspace(xh[0], xh[-1], n_nodes)])
    uh_list = [uh.copy()]

    # Crank–Nicolson matrices
    A = M + 0.5 * dt * K
    B = M - 0.5 * dt * K

    for n in range(1, N):
        t_n = time[n - 1]
        t_np1 = time[n]

        F_n = F_func(t_n)
        F_np1 = F_func(t_np1)
        F_avg = 0.5 * (F_n + F_np1)

        rhs = B @ uh + dt * F_avg

        # Enforce Dirichlet BC at node 0: u(0,t) = p0
        A = A.tolil()
        rhs[0] = p0
        A[0, :] = 0
        A[0, 0] = 1

        # Solve for next time step
        uh = spsolve(A.tocsr(), rhs)
        uh_list.append(uh.copy())

    return uh_list, time


In [ ]:
# This file contains functions that assemble calculated node values
# from a 1D FEM menthod to approximated solution
#
#
# Last update: Chung-min Lee March 15, 2025

import numpy as np
!!!!! NOTE THAT YOU NEED TO IMPORT SHAPEFNS !!!!

##--------------------------------------------------
# assemble solution on the nodes and give the solution function
##--------------------------------------------------
def approxSol(w: np.array, p0: float, xh: np.array, shapeFn:int):
    '''
        This function takes the solution of the values of the nodes and gives the
        approximated solution on the interval [0,L]
        [uh] = approxSol(w, u0, xh, shapeFn)
        where
            w is the values of the nodes excluding the node value at x=0,
            p0 is the Dirichlet BC at x = 0,
            xh = [x_1,x_2,...,x_{n+1}] is the list of endpoints of all elements,
            shapeFn indicates the choice of linear (use 1) or quadratic (use 2) shape
            functions
    '''
    w = np.hstack((p0, w))
    if (shapeFn == 1): # if linear shape functions are used
        uh = lambda x: linearApprox(x,w,xh)
    else: # if quadratic shape functions are used
        uh = lambda x: quadraticApprox(x,w,xh)
    return uh

##--------------------------------------------------
# assemble linear shape functions on the whole domain
##-------------------------------------------------
def linearApprox(x: np.array, v: np.array,xh: np.array):
    if (np.ndim(x) == 0):
        #print('x is a scalar')
        x = np.array([x])
    
    # declare the output array
    y = np.zeros(len(x))
    
    # mesh size
    h = xh[1]-xh[0]
    
    # determine which subinterval x belongs to
    ind = np.floor(x/h).astype(int)
    x1 = h * ind
    x2 = x1 + h
    ind = ind+1
    
    # the right end point x=L
    indLastPt = np.where(ind == len(v))[0]
    y[indLastPt] = v[-1]
    indNotLastPt = np.where(ind < len(v))[0]
    y[indNotLastPt] = v[ind[indNotLastPt]-1] * shapeFn1d(1,x[indNotLastPt],x1[indNotLastPt],x2[indNotLastPt],1) + \
    v[ind[indNotLastPt]] * shapeFn1d(2,x[indNotLastPt],x1[indNotLastPt],x2[indNotLastPt],1)
    return y

##--------------------------------------------------
# assemble quadratic shape functions on the whole domain
##-------------------------------------------------
def quadraticApprox(x: np.array,v: np.array,xh: np.array):
    if (np.ndim(x) == 0):
        #print('x is a scalar')
        x = np.array([x])
    
    # declare the output array
    y = np.zeros(len(x))
    
    # mesh size
    h = xh[1]-xh[0]
    
    # determine which subinterval x belongs to
    ind = np.floor(x/h).astype(int)
    x1 = ind*h
    x2 = x1+h
    ind = ind+1
    
    # the right end point x=L
    indLastPt = np.where(ind == (len(v)+1)/2)[0]
    y[indLastPt] = v[-1]
    indNotLastPt = np.where(ind < (len(v)+1)/2)[0]
    y[indNotLastPt] = v[2*ind[indNotLastPt]-2] * shapeFn1d(1,x[indNotLastPt],x1[indNotLastPt],x2[indNotLastPt],2)\
    + v[2*ind[indNotLastPt]-1] * shapeFn1d(2,x[indNotLastPt],x1[indNotLastPt],x2[indNotLastPt],2) \
    + v[2*ind[indNotLastPt]] * shapeFn1d(3,x[indNotLastPt],x1[indNotLastPt],x2[indNotLastPt],2)
    return y


In [18]:
def myFE1dibvp(a, c, f, p0, QL, u0, L, T, dt, noOfEle, shapeFn):
    xh = np.linspace(0, L, noOfEle + 1)
    #M = massM(xh, shapeFn, 3)
    #K = stiffK(a, c, xh, shapeFn, 3)
    #Ft = loadF(a, c, f, p0, QL, xh, shapeFn, 3)
    
    #uh = approxSol(Wn, p0, xh, shapeFn)
    uh = crank_nicolson_solver(xh, a, c, f, p0, QL, shapeFn, 3, u0, 0, T, dt)
    
    return uh
    

In [21]:
#Test case 1
import numpy as np
a = lambda x: 4-x
c = lambda x: 1*x
f = lambda x,t:-2*t*np.sin(x)/((1+t**2)**2) + (np.cos(x)+4*np.sin(x))/(1+t**2)
L = 2.
T = 1.
p0 = 0.
QL = lambda t: 2*np.cos(2)/(1+t**2)
u0 = lambda x: np.sin(x)
dt = 0.1
shapeFn = 1
noOfEle = 4
#uh,W = myFE1dibvp(a, c, f, p0, QL, u0, L, T, dt, noOfEle, shapeFn)
uh = myFE1dibvp(a, c, f, p0, QL, u0, L, T, dt, noOfEle, shapeFn)
np.set_printoptions(precision=4)
#print(f'W = {W}')
print(f'uh(x) = {uh[5](np.array([0.2, 1.37, 0.15, 2.22]))}') 

[0.40943074 0.55966456]
[0.80910176 0.90206472]
[1.01067646 1.02360798]
[0.9648023 0.8945363]
[0.39730934 0.53818989]
[0.77099739 0.85709995]
[0.9559184  0.96616204]
[0.90679724 0.83867398]


ValueError: dimension mismatch

In [14]:
#Test case 2
shapeFn = 2
uh = myFE1dibvp(a, c, f, p0, QL, u0, L, T, dt, noOfEle, shapeFn)
np.set_printoptions(precision=4)
#print(f'W = {W}')
print(f'uh(x) = {uh[5](np.array([1.2, 0, 2.15, 0.49]))}')

NameError: name 't' is not defined

In [ ]:
#Project 1 problem
import numpy as np
a = lambda x: 2 + x
c = lambda x: 1*x
f = lambda x,t:(2 + 2*t - 4*x*t - 4*x**2*t)*np.cos(x) + (1 - 8*t - 6*x*t)*np.sin(x)
L = np.pi
T = 1.
p0 = 1.
QL = lambda t: 2*t*(np.pi + 2)
u0 = lambda x: np.cos(x)
dt = 0.01
shapeFn1 = 1
shapeFn2 = 2
noOfEle1 = 16
noOfEle2 = 32
noOfEle3 = 64
noOfEle4 = 128

true_soln = lambda x,t: (1-2*x*t)*np.cos(x)

lin_uh16 = myFE1dibvp(a, c, f, p0, QL, u0, L, T, dt, noOfEle1, shapeFn1)
lin_uh32 = myFE1dibvp(a, c, f, p0, QL, u0, L, T, dt, noOfEle2, shapeFn1)
lin_uh64 = myFE1dibvp(a, c, f, p0, QL, u0, L, T, dt, noOfEle3, shapeFn1)
lin_uh128 = myFE1dibvp(a, c, f, p0, QL, u0, L, T, dt, noOfEle4, shapeFn1)

quad_uh16 = myFE1dibvp(a, c, f, p0, QL, u0, L, T, dt, noOfEle1, shapeFn2)
quad_uh32 = myFE1dibvp(a, c, f, p0, QL, u0, L, T, dt, noOfEle2, shapeFn2)
quad_uh64 = myFE1dibvp(a, c, f, p0, QL, u0, L, T, dt, noOfEle3, shapeFn2)
quad_uh128 = myFE1dibvp(a, c, f, p0, QL, u0, L, T, dt, noOfEle4, shapeFn2)

x_vals = np.linspace(0, L, 100)
np.set_printoptions(precision=4)

In [ ]:
plt.plot(x_vals, true_soln(x_vals,1), label='true solution')
plt.plot(x_vals, lin_uh16[-1](x_vals), label='16 elements')
plt.plot(x_vals, lin_uh32[-1](x_vals), label='32 elements')
plt.plot(x_vals, lin_uh64[-1](x_vals), label='64 elements')
plt.plot(x_vals, lin_uh128[-1](x_vals), label='128 elements')

plt.xlabel('x')
plt.ylabel('u(x, 1)')
plt.title('FEM solution at t=1 for different mesh resolutions')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
plt.plot(x_vals, true_soln(x_vals,1), label='true solution')
plt.plot(x_vals, quad_uh16[-1](x_vals), label='16 elements')
plt.plot(x_vals, quad_uh32[-1](x_vals), label='32 elements')
plt.plot(x_vals, quad_uh64[-1](x_vals), label='64 elements')
plt.plot(x_vals, quad_uh128[-1](x_vals), label='128 elements')

plt.xlabel('x')
plt.ylabel('u(x, 1)')
plt.title('FEM solution at t=1 for different mesh resolutions')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
plt.plot(x_vals, np.abs(true_soln(x_vals,1) - quad_uh16[-1](x_vals)), label='16 elements')
plt.plot(x_vals, np.abs(true_soln(x_vals,1) - quad_uh32[-1](x_vals)), label='32 elements')
plt.plot(x_vals, np.abs(true_soln(x_vals,1) - quad_uh64[-1](x_vals)), label='64 elements')
plt.plot(x_vals, np.abs(true_soln(x_vals,1) - quad_uh128[-1](x_vals)), label='128 elements')

plt.xlabel('x')
plt.ylabel('absolute difference at x')
plt.title('Absolute difference between FEM and true solution at t=1 for different mesh resolutions')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
x_vals = np.linspace(0, L, 100)           # 100 spatial points
t_vals = np.arange(0, T + dt, dt)         # All time steps including final time
X, T_grid = np.meshgrid(x_vals, t_vals)   # For surface plotting

def u_exact(x, t):
    return (1 - 2*x*t)*np.cos(x)           # Replace this with your actual solution

# Get your computed uh from the solver
uh = myFE1dibvp(a, c, f, p0, QL, u0, L, T, dt, noOfEle=128, shapeFn=2)

# Allocate array to store the error
abs_error = np.zeros_like(X)

# Loop through time steps and compute absolute error
for i, t in enumerate(t_vals):
    uh_t = uh[i](x_vals)                 # FEM solution at time t
    u_true = u_exact(x_vals, t)         # Exact solution
    abs_error[i, :] = np.abs(uh_t - u_true)

from mpl_toolkits.mplot3d import Axes3D

fig = plt.figure(figsize=(10, 6))
ax = fig.add_subplot(111, projection='3d')
ax.plot_surface(X, T_grid, abs_error, cmap='viridis')
ax.set_xlabel('x')
ax.set_ylabel('t')
ax.set_zlabel('Absolute Error')
ax.set_title('Absolute Error |u_exact - u_h| over (x, t)')
plt.tight_layout()
plt.show()


In [ ]:
A = np.array([[1, np.log(16)], [1, np.log(32)], [1, np.log(64)], [1, np.log(128)]])
b = np.array([np.linalg.norm(true_soln(x_vals,1) - quad_uh16[-1](x_vals)), np.linalg.norm(true_soln(x_vals,1) - quad_uh32[-1](x_vals)), np.linalg.norm(true_soln(x_vals,1) - quad_uh64[-1](x_vals)), np.linalg.norm(true_soln(x_vals,1) - quad_uh128[-1](x_vals))])
x, residuals, rank, s  = np.linalg.lstsq(A,b, rcond = None)
print(x)